#### 1. Importing Libraries and Modules

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim as gs
#import slugify as sg
import nltk
from re import sub # import sub to replace items in the followiong list comprehension
from collections import defaultdict
from sklearn.lda import LDA


C:\Users\chris\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


#### 2. Read Data, Split into "Train", "Test", "Validate"

In [2]:
data = pd.read_table('SMSSpamCollection',header= None, names = ('outcome', 'content'))

In [3]:
n = 3000
train = data.sample(n)
test = data[~data.index.isin(train.index)]
validate = test.sample(1000)
test= test[~test.index.isin(validate.index)]
split_correctly = 0 == sum(validate.index.isin(test.index)) + sum(test.index.isin(train.index)) + sum(validate.index.isin(train.index))
set_n_sizes = 'N\'s in .. train:', train.shape,'test:', test.shape,'validate:', validate.shape

print(
    'Data Split Correct?',
    split_correctly,
    '\n'*2,
    set_n_sizes)

Data Split Correct? True 

 ("N's in .. train:", (3000, 2), 'test:', (1572, 2), 'validate:', (1000, 2))


#### 3. Data Preprocessing

In [4]:
def prep_nlp(data_to_prep, stop_words_in):
    # lower case it
    clean = list(data_to_prep.str.lower())
    # this will tokenize
    clean = [[word for word in document.split()] for document in clean]
    #stopwords_set1 = set(nltk.corpus.stopwords.words('english'))
    words_to_remove = '|'.join(stopwords_set3)
    symbol_remover = '[^A-Za-z0-9]+'
    clean = [[sub(symbol_remover,'',word) for word in text] for text in clean]
    clean = [[sub(words_to_remove,'',word) for word in text] for text in clean]
    return clean

In [5]:
stopwords_set2 = set('for a of the and to in or'.split())
stopwords_set3 = ''
train_prepped = prep_nlp(data_to_prep = train.content, stop_words_in= stopwords_set2)

#### 4. Build unsupervised Model

In [6]:
def build_model(train_data, topic_n):
    frequency = defaultdict(int)
    for text in train_data:
        for token in text:
            frequency[token] += 1
    # get freq > 1
    word_freq_1plus = [[x for x in words if frequency[x] > 1] for words in train_data]
    # Create dictionary
    dictionary = gs.corpora.Dictionary(word_freq_1plus)
    # Create Corpus
    corpus = [dictionary.doc2bow(text) for text in train_data]
    # corpus to tfidf
    tfidf = gs.models.TfidfModel(corpus) 
    corp_tf = tfidf[corpus] 
    # Unsupervised Component. Reduce space into 300 topics. 
    topic_n = topic_n
    lsi = gs.models.LsiModel(corp_tf, id2word=dictionary, num_topics = topic_n)
    corp_topics = lsi[corp_tf] 
    return corp_topics, dictionary, tfidf, lsi    

In [7]:
built_model = build_model(train_data = train_prepped, topic_n = 300)

#### 5. Build supervised Model

In [8]:
def train_model(topic_vec):
    x = pd.DataFrame([dict(row) for row in topic_vec[0]])
    y = (train["outcome"] == "spam").astype(int) 
    lda = LDA()
    mask = np.array([~np.isnan(row).any() for row in x.values])
    x_masked = x[mask]
    y_masked = y[mask]
    lda = lda.fit(x_masked,y_masked)
    return lda,x_masked,y_masked, topic_vec[1],topic_vec[2], topic_vec[3]

In [67]:
trained_model = train_model(topic_vec  = built_model)
sum(trained_model[0].predict(trained_model[1])==trained_model[2])/len(trained_model[2]) 

0.98030050083472453

In [10]:
trained_model[3]

#### 6. Test Model on Unseen Data

In [71]:
test.head()

,outcome,content
3,ham,U dun say so early hor... U c already then say...
6,ham,Even my brother is not like to speak with me. ...
9,spam,Had your mobile 11 months or more? U R entitle...
14,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
16,ham,Oh k...i'm watching here:)


In [73]:
test.content.head()

3     U dun say so early hor... U c already then say...
6     Even my brother is not like to speak with me. ...
9     Had your mobile 11 months or more? U R entitle...
14                  I HAVE A DATE ON SUNDAY WITH WILL!!
16                           Oh k...i'm watching here:)
Name: content, dtype: object

In [97]:
def predict_unseen(new_doc_in, stop_words_in, trained_model_in):
    
    dictionary_in = trained_model_in[3]
    tfidf_in = trained_model_in[4]
    lsi_in = trained_model_in[5]
    lda_in = trained_model_in[0]
    new_doc_in_content = pd.Series(new_doc_in.content)
    new_doc_in_outcome = pd.Series(new_doc_in.outcome)
    
    query = prep_nlp(new_doc_in_content, stop_words_in= stop_words_in)
    query_bow = [dictionary_in.doc2bow(corp) for corp in query]
    query_tf = tfidf_in[query_bow] 
    
    x_2 = pd.DataFrame([dict(tf) for tf in lsi_in[query_tf]])
    mask = np.array([~np.isnan(row).any() for row in x_2.values])
    x_2masked = x_2[mask]
    y_2 = (new_doc_in_outcome == "spam").astype(int) 
    
    y_2masked = np.array(y_2[mask])
    x_2masked = lda_in.predict(x_2masked)
    
    return x_2masked,y_2masked


In [115]:
new_doc = ["hey dude where are you",
           "text 444 for a promotional treat",
           "dont know what time it is", 
           "Our records indicate your Pension is under performing to see higher growth and up to 25% cash release reply PENSION for a free review. To opt out reply STOP",
          "To start the process please reply YES. To opt out text STOP",
          "i'm going to be 10 mins late"]
new_doc_results = ['ham','spam','ham','spam','spam','ham']
external_data = pd.DataFrame({'content':new_doc, 'outcome':new_doc_results})

predicted_test = predict_unseen(new_doc_in=test, stop_words_in = stopwords_set2, trained_model_in = trained_model)
predicted_external = predict_unseen(new_doc_in=external_data, stop_words_in = stopwords_set2, trained_model_in = trained_model)

##### 7.Performance
There are three performance metrics:
1. "Accuracy" which tells us, what percent of predicted results equal the actual results
2. "Precision": Of all all observations we predicted as spam, what is actually spam?
3. "Recall": Of all observations actually spam, what percent did we predict?

In [116]:
def performance(result_x, result_y):
    
    actual_positive = result_y == 1
    actual_negative = result_y ==0
    
    true_positives = result_x[actual_positive] == 1
    false_positives = result_x[actual_negative] == 1
    true_negatives = result_x[actual_negative] == 0
    false_negatives = result_x[actual_positive] == 0
    
    #A. Accuracy = (TP + TN)/(TP + TN + FP + FN)
    #B. Precision = TP/(TP + FP)
    #C. Recall = TP/(TP + FN)
    accuracy = sum((result_x == result_y))/len(result_y)
    precision = sum(true_positives) / (sum(true_positives) + sum(false_positives))
    recall = sum(true_positives) / (sum(true_positives) + sum(false_negatives))
    return [accuracy, precision, recall, len(result_x)]

In [106]:
len(predicted_test[1])


1568

In [117]:
performance_on_train = performance(result_x=trained_model[0].predict(trained_model[1]),result_y=trained_model[2])
performance_on_test = performance(result_x=predicted_test[0],result_y=predicted_test[1])
performance_on_external = performance(result_x=predicted_external[0],result_y=predicted_external[1])


In [119]:

x = pd.DataFrame({
    'Train':performance_on_train,
    'Test':performance_on_test,
    'external':performance_on_external

}).set_index(
    [['Accuracy','Precision','Recall','N Size'],
     ['% Spam / Ham Correct','% Predicted Spam Actually Spam','% Spam Detected','']])
x

,,Test,Train,external
Accuracy,% Spam / Ham Correct,0.977041,0.980301,1.0
Precision,% Predicted Spam Actually Spam,0.965116,0.977961,1.0
Recall,% Spam Detected,0.846939,0.874384,1.0
N Size,,1568.000000,2995.000000,6.0
